## **데이터 업로드**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import matplotlib.pyplot as plt 
import platform                

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus']= False

if platform.system() == 'Darwin': #맥os 사용자의 경우
    plt.style.use('seaborn-darkgrid') 
    rc('font', family = 'AppleGothic')
    
elif platform.system() == 'Windows':#윈도우 사용자의 경우
    path = 'c:/Windows/Fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname=path).get_name()
    plt.style.use('seaborn-darkgrid') # https://python-graph-gallery.com/199-matplotlib-style-sheets/
    rc('font', family=font_name)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

ParserError: ignored

## **0) 목적: 리뷰 평점 바탕 추천시스템**

> 그핵이는 '와인 마켓'을 운영하는데, 정교한 선호도를 바탕으로 유저들에게 평점이 높은 와인 순서대로 추천하고 싶다. 어떤 변수를 활용하면 가장 정확하게 평점을 예측해볼 수 있을까?

> 월 1회 프로모션 이벤트를 진행하고 싶은데, 어떤 컨셉(특정 지역, 특정 분류의 와인)으로 진행했을 때 구매 유도에 효과적일까?

## **1) EDA 및 데이터 파악**
  - 분포 확인
  - 결측치, 이상치 파악

In [ ]:
train.head()

In [ ]:
train.info()

> 평가자의 주관적 평가(description)를 평가의 단어 개수(decription_count로) 대체

In [ ]:
word_count_train = []
for i in range(len(train)):
  word_count_train.append(train.description[i].count(" ") + 1)

train['description_count'] = word_count_train

> Title 변수 안에 있는 Year 정보 추출

In [ ]:
import re
year_list = []    
for i in train['title']:
    year = re.search(r'19\d{2}|20\d{2}', i)
    if year:
        year_list.append(year.group())
    else: 
      year_list.append(None)

train ['year'] = year_list

> 결측치 확인 및 이상치 확인

In [ ]:
train.isnull().sum()/len(train)*100

> 분포 확인

In [ ]:
train.describe()

In [ ]:
plt.figure(figsize = (10,5))
train_eda = train.dropna(axis = 0)

price = sns.distplot(train_eda['points'])
price.set_xlabel('points')
price.set_ylabel('per')
plt.show

print('skewness: %f' % train_eda['points'].skew()) 
print('kurtosis: %f' % train_eda['points'].kurt()) 

In [ ]:
plt.figure(figsize = (10,5))
train_eda = train.dropna(axis = 0)

price = sns.distplot(train_eda['description_count'])
price.set_xlabel('count')
price.set_ylabel('per')
plt.show

In [ ]:
plt.figure(figsize = (10,5))
train2 = train.dropna(axis = 0)

price = sns.distplot(train['price'])
price.set_xlabel('price($)')
price.set_ylabel('per')
plt.show

## **2) 데이터 시각화**

> 1.와인 품질 

> [참고: 와인 품질을 결정하는 요소](https://www.dongacc.com/cooperation/contents/wineschool/1_08selection.htm)
 - 포도의 종류
 - 포도의 생산지
 - 생산년도
 - 제조기술 
 
> 포도의 종류를 나타내는 **variety** / 생산지를 나타내는 **country, province** / 제조기술을 보여주는는 **winery** / 생산년도를 나타내는 **year(title에서 추출)** 변수 활용 가능



> 2. 평가자의 성향
 - **taster name** 
 - **description**의 단어 개수 활용
 - 긍정어? 감정 분석

> 3. 가격 (**price**)

> 상관관계 확인

In [ ]:
train.corr()

> Price

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(train['price'],
            train['points'], 
            ) 
plt.ylabel('points')
plt.xlabel('price')
plt.xticks(rotation=90) 
plt.show()

> Description_count

In [ ]:
plt.figure(figsize = (10,5))
sns.regplot(data = train, x= 'description_count', y= 'points')
plt.xlabel('description_count')
plt.ylabel('points')
plt.ylim([80,100])
plt.show

> WInery
- 데이터수가 많은 상위 30개의 winery만 추출. 

In [ ]:
winery_top30 = train.groupby('winery').size().reset_index(name = 'n').sort_values('n', ascending  = False).reset_index(drop = True).head(30)
index30_winery = train[~(train['winery'].isin(list(winery_top30['winery'])))].index
train30_winery = train.drop(index30_winery)

In [ ]:
plt.figure(figsize = (20,8))
box = sns.boxplot(x='winery', y= 'points', data = train30_winery)
plt.show()

> Province 

In [ ]:
province_top30 = train.groupby('province').size().reset_index(name = 'n').sort_values('n', ascending  = False).reset_index(drop = True).head(30)
index30_province = train[~(train['province'].isin(list(province_top30['province'])))].index
train30_province = train.drop(index30_province)

In [ ]:
plt.figure(figsize = (20,8))
box = sns.boxplot(x='province', y= 'points', data = train30_province)
plt.show()

> Price vs Points (top10 winery) 

In [ ]:
winery_top10 = train.groupby('winery').size().reset_index(name = 'n').sort_values('n', ascending  = False).reset_index(drop = True).head(10)
index10_winery = train[~(train['winery'].isin(list(winery_top10['winery'])))].index
train10_winery = train.drop(index10_winery)

In [ ]:
plt.figure(figsize=(15,6))
sns.scatterplot(train10_winery['price'], 
            train10_winery['points'], 
            hue=train10_winery['winery'] 
            ) 
plt.ylabel('points')
plt.xlabel('price')
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(15,6))
sns.scatterplot(train10_winery['price'], 
            train10_winery['points'], 
            hue=train10_winery['winery'] 
            ) 
plt.title('Price vs Points (by top10 winery)')
plt.ylabel('points')
plt.xlabel('price')
plt.xlim([0,100])
plt.xticks(rotation=90)
plt.show()

> Price vs Points (top10 province)

In [ ]:
province_top10 = train.groupby('province').size().reset_index(name = 'n').sort_values('n', ascending  = False).reset_index(drop = True).head(10)
index10_province = train[~(train['province'].isin(list(province_top10['province'])))].index
train10_province = train.drop(index10_province)

In [ ]:
plt.figure(figsize=(15,6))
sns.scatterplot(train10_province['price'], 
            train10_province['points'], 
            hue=train10_province['province'] 
            ) 
plt.title('Price vs Points (by top10 province)')
plt.ylabel('points')
plt.xlabel('price($)')
plt.xlim([0,100])
plt.xticks(rotation=90) 
plt.show()

##California 데이터가 지나치게 많아서 경향성이 잘 보이지 않음

In [ ]:
index_california = train10_province[train10_province['province'] == 'California'].index
train_no_california = train10_province.drop(index_california)

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(train_no_california['price'], 
            train_no_california['points'], 
            hue=train_no_california['province'] 
            ) 
plt.title('Price vs Points (by top10 province)')
plt.ylabel('points')
plt.xlabel('price($)')
plt.xlim([0,500])
plt.xticks(rotation=90) 
plt.show()

> 인사이트 및 활용 방안 정리

## **3) 데이터 전처리**
- 결측치 처리
- categorical 데이터 전처리
- train/validation/test set 분리

> 결측치 처리

In [ ]:
train.isnull().sum()/len(train) *100
## 열 제거(활용하지 않을 변수): designation, region_1, region_2, taster_twitter_handle
## 행 제거: country, variety, province
## 'None'값 대체: taster_name
## mean 값 대체: price

In [ ]:
train1 = train.drop(['designation', 'region_1', 'regioni_2', 'taster_twitter_handle'], axis = 1)
train1['taster_name'] = train['taster_name'].fillna('None')
train1['price'] = train1['price'].fillna(train1['price'].mean())
train_final = train1.dropna(axis = 0)

> train / validation / test set 분리

In [ ]:
# predictor variable과 target variable 분리
X = train_final.copy()
y = X['points']
X.drop(['points'], axis=1, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

> Categorical 데이터 전처리 (One-hot-encoding)

## **4) 모델링 및 Target 예측**